<a href="https://colab.research.google.com/github/Snjkmr162/BLS_Chatbot_with_RL/blob/main/Chatbot_with_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs & Imports

In [1]:
!pip install requests pandas matplotlib scikit-learn transformers torch

In [2]:
import requests
import pandas as pd
import numpy as np
import pickle
import os
import random
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# BLS Data Fetching & Loading

In [3]:
def fetch_bls_series(series_id, start_year, end_year):
    url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
    payload = {
        "seriesid": [series_id],
        "startyear": start_year,
        "endyear": end_year
    }

    response = requests.post(url, json=payload).json()
    data = response["Results"]["series"][0]["data"]

    df = pd.DataFrame(data)
    df["year"] = df["year"].astype(int)
    df["value"] = df["value"].astype(float)
    return df.sort_values(["year", "period"])

In [4]:
# Unemployment rate
unemployment_df = fetch_bls_series("LNS14000000", "2019", "2024")

# CPI for inflation
cpi_df = fetch_bls_series("CUUR0000SA0", "2019", "2024")

def compute_inflation_rate(cpi_df):
    df = cpi_df.copy()
    df["inflation_rate"] = df["value"].pct_change(periods=12) * 100
    return df.dropna()

inflation_df = compute_inflation_rate(cpi_df)

# Rule Based Explanations

In [5]:
def explain_unemployment():
    peak = unemployment_df["value"].max()
    latest = unemployment_df.iloc[-1]["value"]
    return (
        f"U.S. unemployment peaked at {peak:.1f}% during COVID and has since "
        f"recovered to about {latest:.1f}%, reflecting gradual labor market recovery."
    )

def explain_inflation():
    latest = inflation_df.iloc[-1]["inflation_rate"]
    peak = inflation_df["inflation_rate"].max()

    return (
        f"U.S. inflation rose sharply after COVID due to supply chain disruptions, "
        f"strong consumer demand, labor shortages, and rising energy costs. "
        f"It peaked at around {peak:.1f}% and has since eased to roughly {latest:.1f}%, "
        "though overall price levels remain elevated."
    )

def explain_comparison():
    return (
        "Inflation and employment are linked through economic cycles. "
        "When unemployment falls, household income and demand often rise, "
        "which can put upward pressure on prices. This relationship is often "
        "described by the Phillips Curve, though supply shocks and policy "
        "responses can weaken or reverse this trade-off."
    )


def auto_reward(intent, confidence, action):
    # Rejecting when unsure is good
    if action == 0 and confidence < 0.4:
        return 0.5

    # Answering confidently is very good
    if action == 1 and confidence >= 0.6:
        return 1.0

    # Medium confidence answer
    if action == 1 and 0.4 <= confidence < 0.6:
        return 0.5

    # Rejecting despite high confidence is bad
    if action == 0 and confidence >= 0.6:
        return -1.0

    # Answering UNKNOWN intent is bad
    if intent == "UNKNOWN" and action == 1:
        return -1.0

    # 🚨 DEFAULT reward (never None)
    return 0.0


# ML Intent Classifier

In [6]:
training_sentences = [
    # Unemployment
    "why did unemployment rise",
    "why did unemployment spike during covid",
    "job losses in 2020",
    "mass layoffs",
    "are jobs recovering",
    "is unemployment falling",
    "labor market recovery",
    "unemployment rate",

    # Inflation
    "what is inflation",
    "inflation rate",
    "are prices rising",
    "why are prices high",
    "why did prices increase",
    "cost of living increase",
    "inflation after covid",
    "is inflation cooling",

    # Comparison
    "relationship between inflation and unemployment",
    "jobs and prices connection",
    "how are jobs and prices connected",
    "inflation vs unemployment",
    "can inflation and unemployment both be high"

]

training_labels = (
    ["UNEMPLOYMENT"] * 8 +
    ["INFLATION"] * 8 +
    ["COMPARISON"] * 5
)


vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(training_sentences)

intent_model = LogisticRegression()
intent_model.fit(X, training_labels)

LogisticRegression()

In [7]:
def detect_intent_with_confidence(text):
    text_l = text.lower()

    # 1️⃣ HARD OVERRIDE FIRST (beats ML)
    relationship_keywords = [
    "relationship",
    "connected",
    "connection",
    "trade-off",
    "trade off",
    "both be high",
    "jobs and prices",
    "inflation employment",
    "inflation and employment",
    "inflation and unemployment",
    "jobs and inflation",
    "prices and jobs",
    "while unemployment"
]

    if any(k in text_l for k in relationship_keywords):
        vec = vectorizer.transform([text])
        probs = intent_model.predict_proba(vec)[0]
        confidence = max(probs)
        return "COMPARISON", confidence

    # 2️⃣ ML prediction only if no override
    vec = vectorizer.transform([text])
    probs = intent_model.predict_proba(vec)[0]

    intent = intent_model.classes_[np.argmax(probs)]
    confidence = np.max(probs)

    if confidence < 0.4:
        return "UNKNOWN", confidence

    return intent, confidence

# LLM

Load LLM

In [8]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Fact Builder

In [9]:
def build_facts(intent, question):
    q = question.lower()

    # -------------------------
    # COMPARISON (already good)
    # -------------------------
    if intent == "COMPARISON":
        return explain_comparison()

    # -------------------------
    # INFLATION
    # -------------------------
    if intent == "INFLATION":

        if any(k in q for k in ["why", "cause", "reason"]):
            return (
                "Inflation rose due to supply chain disruptions, strong consumer demand, "
                "labor shortages, and higher energy and food costs following the pandemic."
            )

        if any(k in q for k in ["easing", "cooling", "coming down"]):
            return (
                "Inflation has eased from its 2022 peak as supply chains normalized "
                "and monetary policy tightened, though prices remain high."
            )

        if any(k in q for k in ["still rising", "high", "expensive", "groceries"]):
            return (
                "Prices remain high because many costs—such as housing, food, and energy—"
                "have not fallen even as inflation slows."
            )

        return explain_inflation()

    # -------------------------
    # UNEMPLOYMENT
    # -------------------------
    if intent == "UNEMPLOYMENT":

        if any(k in q for k in ["spike", "layoff", "loss", "covid"]):
            return (
                "Unemployment spiked during COVID due to lockdowns, business closures, "
                "and a sudden collapse in economic activity."
            )

        if any(k in q for k in ["recover", "improving", "finding jobs", "down"]):
            return (
                "The labor market has recovered as businesses reopened and hiring resumed, "
                "bringing unemployment back down to around 4.1%."
            )

        return explain_unemployment()

    # -------------------------
    # FALLBACK
    # -------------------------
    return (
        "I’m not fully confident about that question, "
        "but I can explain inflation, unemployment, or how they’re related."
    )

LLM Response

In [10]:
def generate_llm_response(question, facts):
    prompt = f"""
You are a U.S. labor economics assistant.
Facts:
{facts}

Question: {question}
Answer clearly and concisely:
"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = llm_model.generate(**inputs, max_length=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# RL

Reinforcement Learning Setup

In [11]:
confidence_bins = np.linspace(0, 1, 11)
Q_table = np.zeros((10, 2))

learning_rate = 0.1
discount_factor = 0.9

epsilon = 0.5
epsilon_min = 0.05
epsilon_decay = 0.995

reward_history = []

Load saved Q-table

In [12]:
if os.path.exists("q_table.pkl"):
    with open("q_table.pkl", "rb") as f:
        Q_table = pickle.load(f)

RL Helpers

In [13]:
def get_confidence_bin(confidence):
    return min(int(confidence * 10), 9)

def choose_action(confidence):
    state = get_confidence_bin(confidence)
    if random.random() < epsilon:
        return random.choice([0, 1])
    return np.argmax(Q_table[state])

def update_q_table(state, action, reward):
    Q_table[state, action] += learning_rate * (
        reward + discount_factor * np.max(Q_table[state]) - Q_table[state, action]
    )

In [14]:
print(build_facts("INFLATION", "why are prices high"))
print(auto_reward("INFLATION", 0.52, 1))

Inflation rose due to supply chain disruptions, strong consumer demand, labor shortages, and higher energy and food costs following the pandemic.
0.5


In [15]:
episode_rewards = []

# Chatbot Loop

In [ ]:
print("📊 BLS Labor Market Chatbot")
print("Ask me about unemployment, inflation, or trends.")
print("Type 'exit' to quit.\n")

while True:
    user_input = input("You: ").strip()

    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break

    intent, confidence = detect_intent_with_confidence(user_input)
    state = get_confidence_bin(confidence)
    action = choose_action(confidence)

    if action == 0 or intent == "UNKNOWN":
        print("Chatbot: I’m not fully confident about that question yet.")
    else:
        facts = build_facts(intent, user_input)
        response = generate_llm_response(user_input, facts)
        print("Chatbot:", response)

    reward = auto_reward(intent, confidence, action)
    episode_rewards.append(reward)
    update_q_table(state, action, reward)

    reward_history.append(reward)

    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    print()

Saving the learning

In [17]:
with open("q_table.pkl", "wb") as f:
    pickle.dump(Q_table, f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

window = 10
smoothed_rewards = np.convolve(
    reward_history, np.ones(window)/window, mode="valid"
)

plt.figure()
plt.plot(smoothed_rewards)
plt.xlabel("Interaction")
plt.ylabel("Average Reward")
plt.title("RL Learning Curve (Smoothed)")
plt.show()

# Chatbot core

In [19]:
epsilon = 0.0  # no exploration for demo

In [27]:
%%writefile chatbot_core.py
import numpy as np
import pandas as pd
import pickle

# -------------------------
# LOAD DATA
# -------------------------
inflation_df = pd.read_csv("inflation_df.csv")
unemployment_df = pd.read_csv("unemployment_df.csv")

# -------------------------
# LOAD INTENT MODEL
# -------------------------
with open("intent_model.pkl", "rb") as f:
    intent_model = pickle.load(f)

with open("vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

# -------------------------
# LOAD RL Q-TABLE
# -------------------------
with open("q_table.pkl", "rb") as f:
    Q_table = pickle.load(f)

# -------------------------
# RL HELPERS
# -------------------------
def get_confidence_bin(confidence):
    if confidence < 0.4:
        return 0
    elif confidence < 0.7:
        return 1
    return 2

def choose_action(confidence):
    state = get_confidence_bin(confidence)
    return np.argmax(Q_table[state])

# -------------------------
# INTENT DETECTION
# -------------------------
def detect_intent_with_confidence(text):
    vec = vectorizer.transform([text])
    probs = intent_model.predict_proba(vec)[0]

    intent = intent_model.classes_[np.argmax(probs)]
    confidence = np.max(probs)

    if confidence < 0.4:
        return "UNKNOWN", confidence

    return intent, confidence

# -------------------------
# FACT BUILDERS
# -------------------------
def explain_inflation():
    latest = inflation_df.iloc[-1]["inflation_rate"]
    peak = inflation_df["inflation_rate"].max()
    return (
        f"U.S. inflation peaked near {peak:.1f}% and has since eased to around "
        f"{latest:.1f}%, though price levels remain elevated."
    )

def explain_unemployment():
    latest = unemployment_df.iloc[-1]["value"]
    return (
        f"The unemployment rate has fallen to around {latest:.1f}%, "
        "reflecting recovery after the pandemic."
    )

def explain_comparison():
    return (
        "Inflation and employment are linked through economic cycles. "
        "Lower unemployment can increase demand and push prices higher, "
        "a relationship often described by the Phillips Curve."
    )

def build_facts(intent, question):
    q = question.lower()

    if ("inflation" in q or "prices" in q) and ("jobs" in q or "unemployment" in q):
        return explain_comparison()

    if intent == "INFLATION":
        return explain_inflation()

    if intent == "UNEMPLOYMENT":
        if "covid" in q or "spike" in q or "loss" in q:
            return (
                "Unemployment spiked during COVID due to lockdowns, "
                "business closures, and a sudden collapse in economic activity."
            )
        return explain_unemployment()

    return "General labor market trends vary across economic cycles."

# -------------------------
# RESPONSE GENERATION
# -------------------------
def generate_llm_response(question, facts):
    return facts

Overwriting chatbot_core.py


In [28]:
!sed -n '110,150p' chatbot_core.py

In [29]:
inflation_df.to_csv("inflation_df.csv", index=False)
unemployment_df.to_csv("unemployment_df.csv", index=False)

with open("intent_model.pkl", "wb") as f:
    pickle.dump(intent_model, f)

with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

In [30]:
%%writefile app.py
import streamlit as st

# -------------------------
# IMPORT CHATBOT CORE
# -------------------------
from chatbot_core import (
    detect_intent_with_confidence,
    build_facts,
    generate_llm_response,
    get_confidence_bin
)

# -------------------------
# PAGE CONFIG
# -------------------------
st.set_page_config(
    page_title="U.S. Labor Market Chatbot",
    page_icon="📊",
    layout="centered"
)

# -------------------------
# HEADER
# -------------------------
st.title("📊 U.S. Labor Market Insight Chatbot")

st.markdown("""
This chatbot provides **data-driven insights** into U.S. labor market trends using:

""")

st.divider()

# -------------------------
# SIDEBAR (GUIDED DEMO)
# -------------------------
st.sidebar.header("💡 Try asking:")
st.sidebar.markdown("""
- Is inflation easing in the U.S.?
- Why did unemployment spike during COVID?
- Are jobs recovering after 2020?
- How are inflation and employment related?
""")

# -------------------------
# USER INPUT
# -------------------------
user_input = st.text_input(
    "Ask a question about inflation, unemployment, or labor market trends:"
)

# -------------------------
# LOW CONFIDENCE RESPONSE
# -------------------------
def low_confidence_response():
    return (
        "I’m not fully confident in classifying this question, "
        "but I can share general insights on U.S. inflation and employment trends."
    )

# -------------------------
# MAIN LOGIC
# -------------------------
if user_input:
    intent, confidence = detect_intent_with_confidence(user_input)

    # Decide response
    if intent == "UNKNOWN":
        response = low_confidence_response()
    else:
        facts = build_facts(intent, user_input)
        response = generate_llm_response(user_input, facts)

    # -------------------------
    # DISPLAY ANSWER
    # -------------------------
    st.markdown("### 💬 Chatbot Answer")
    st.write(response)

    # -------------------------
    # CONFIDENCE BAR
    # -------------------------
    st.progress(min(confidence, 1.0))
    st.caption(f"Model confidence: {confidence:.1%}")

    # -------------------------
    # DIAGNOSTICS (COLLAPSIBLE)
    # -------------------------
    with st.expander("🔍 Model Diagnostics"):
        st.write(f"**Detected intent:** {intent}")
        st.write(f"**Confidence score:** {confidence:.3f}")
        st.write(f"**RL confidence bin:** {get_confidence_bin(confidence)}")

        st.caption(
            "Reinforcement Learning is used to learn a response policy "
            "based on confidence levels, balancing informativeness and reliability."
        )

# -------------------------
# FOOTER
# -------------------------
st.divider()
st.caption(
    "Note: Models are trained offline. This application performs real-time inference only."
)

Overwriting app.py


In [23]:
!pip install -q streamlit

In [24]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

In [31]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501

2026-02-04T04:49:29Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-02-04T04:49:29Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-02-04T04:49:34Z INF +--------------------------------------------------------------------------------------------+
2026-02-04T04:49:34Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-02-04T04:49:34Z INF |  https://gazette-lone-volunteer-hydraulic.trycloudflar